## Theoretical model of BF-Max decoder for LDPC codes with fixed column and row weight.

In [2]:
reset()

R = RealField(4000);

def compute_rho_0(n, w, u):
    """
    Compute the probability p_0_uns, i. e. probability distributin of 0-counter.
    
    Parameters:
    n (int): Number of variable nodes (number of columns of H),
    w (int): Row weight of H,
    u (int): Fixed number of errors occurred.
    
    Returns:
    float: Computed probability.
    """
    return sum([R(1.)*binomial(u, ell) * binomial(n - 1 - u, w - 1 - ell) / binomial(n - 1, w - 1) for ell in range(1, min(w - 1, u) + 1, 2)])


def compute_rho_1(n, w, u):
    """
    Compute the probability p_1_uns, i. e. probabillity distribution of 1-counter.
    
    Parameters:
    n (int): Number of variable nodes (number of columns of H),
    w (int): Row weight of H,
    u (int): Fixed number of errors occurred.
    
    Returns:
    float: Computed probability.
    """
    return sum([R(1.)*binomial(u - 1, ell) * binomial(n - u, w - 1 - ell) / binomial(n - 1, w - 1) for ell in range(0, min(w - 1, u - 1) + 1, 2)])


def compute_g0(v, x, rho_0):
    """
    Compute the probability Pr[sigma_i = x | e_i = 0].
    
    Parameters:
    v (int): Column weight of H (number of columns of H),
    x (int): Value of the i-th counter,
    rho_0 (float): Probability distribution of 0-counter.
    
    Returns:
    float: Computed probability.
    """
    return R(1.) * binomial(v, x) * (rho_0 ** x) * ((1 - rho_0) ** (v - x))


def compute_g1(v, x, rho_1):
    """
    Compute the probability Pr[sigma_i = x | e_i = 1].
    
    Parameters:
    v (int): Column weight of H (number of columns of H),
    x (int): Value of the i-th counter,
    rho_1 (float): Probability distribution of 1-counter.
    
    Returns:
    float: Computed probability.
    """
    return R(1.) * binomial(v, x) * (rho_1 ** x) * ((1 - rho_1) ** (v - x))   


def compute_f0_u(rho_0, u, x, n):
    """
    Compute f_0^(u)(x).
    
    Parameters:
    rho_0 (float): Probability distribution of 0-counter,
    u (int): Number of errors occurred,
    x (int): Value of the i-th counter,
    n (int): Number of variable nodes (number of columns of H).
    
    Returns:
    float: Computed probability.
    """
    if x == 0:
        return compute_g0(v, 0, rho_0)^(n-u)
    else:
        pr_max_x_minus_1 = sum([compute_g0(v, z, rho_0) for z in range(x)])
        pr_max_x = pr_max_x_minus_1 + compute_g0(v, x, rho_0)
        return pr_max_x^(n-u) - pr_max_x_minus_1^(n-u)


def compute_f1_u(rho_1, u, x, n):
    """
    Compute f_1^(u)(x).
    
    Parameters:
    rho_1 (float): Probability distribution of 1-counter,
    u (int): Number of errors occurred,
    x (int): Value of the i-th counter,
    n (int): Number of variable nodes (number of columns of H).
    
    Returns:
    float: Computed probability.
    """
    if x == 0:
        return 1 - compute_g1(v, 0, rho_1)^u
    else:
        pr_max_x = sum([compute_g1(v, z, rho_1) for z in range(x+1)])
        return 1 - pr_max_x^u
        
def pr_iteration_ok(n, w, v, u):
    """
    Compute the probability pr_ok_t.
    
    Parameters:
    n (int): Number of variable nodes (number of columns of H),
    w (int): Row weight of H,
    v (int): Column weight of H,
    u (int): Fixed number of errors occurred.
    
    Returns:
    float: Computed probability.
    """
    rho_0 = compute_rho_0(n, w, u)
    rho_1 = compute_rho_1(n, w, u)
    
    pr_ok = 0
    
    for x in range(v):
        #Probability that maximum counter among 0-bits is equal to x
        pr_leq_x_zero_bits = compute_f0_u(rho_0, u, x, n)
        
        #Probability that maximum counter among 1-bits is greater than x
        pr_greater_x_one_bits = compute_f1_u(rho_1, u, x, n)
        
        pr_ok += (pr_leq_x_zero_bits*pr_greater_x_one_bits)
    
    return pr_ok

def compute_dfr(n, w, v, t):
    """
    Compute the complementary probability of DFR.
    
    Parameters:
    n (int): Number of variable nodes (number of columns of H),
    w (int): Row weight of H,
    v (int): Column weight of H,
    t (int): Number of errors occurred.
    
    Returns:
    float: Computed decoding probability.
    """
    pr_decode = R(1)
    for u in range(1, t + 1):
        pr_decode = pr_decode * pr_iteration_ok(n, w, v, u)
    return R(1) - pr_decode


## Tests

In [5]:
# Fixed t and v, increasing r by 100
r_values = range(100, 5100, 100)

v = 17
t = 18
w = 2*v

th_dfr_values = []

print(f"v = {v}, w = {w}, t = {t}")

for r in r_values:
    dfr = compute_dfr(2*r, w, v, t)
    th_dfr_values.append((r, dfr))
    print(f"({r}, {round(dfr, 20)})")

v = 17, w = 34, t = 18
(100, 1.0)
(200, 0.9999999993134756)
(300, 0.9991765327693359)
(400, 0.8141876112239488)
(500, 0.24745358736093723)
(600, 0.038080293261678874)
(700, 0.005133447734468386)
(800, 0.0007373842522775567)
(900, 0.0001179367793120127)
(1000, 2.12165725907571e-05)
(1100, 4.28495584693924e-06)
(1200, 9.5939508451218e-07)
(1300, 2.3481047742548e-07)
(1400, 6.230030449212e-08)
(1500, 1.786792532913e-08)
(1600, 5.53187622549e-09)
(1700, 1.84266099274e-09)
(1800, 6.5621659195e-10)
(1900, 2.4777160679e-10)
(2000, 9.833152643e-11)
(2100, 4.070441766e-11)
(2200, 1.746876369e-11)
(2300, 7.73794234e-12)
(2400, 3.52697614e-12)
(2500, 1.65092773e-12)
(2600, 7.9264008e-13)
(2700, 3.9008406e-13)
(2800, 1.9671814e-13)
(2900, 1.0164789e-13)
(3000, 5.38176e-14)
(3100, 2.919586e-14)
(3200, 1.622724e-14)
(3300, 9.23798e-15)
(3400, 5.38403e-15)
(3500, 3.21015e-15)
(3600, 1.95626e-15)
(3700, 1.2171e-15)
(3800, 7.721e-16)
(3900, 4.9876e-16)
(4000, 3.2764e-16)
(4100, 2.1857e-16)
(4200, 1.478

In [ ]:
# Fixed r and v, decreasing t.
t_values = range(100, 14, -1)

r = 2003
v = 9
w = 2*v

th_dfr_values = []

print(f"v = {v}, w = {w}, r = {r}")

for t in t_values:
    dfr = compute_dfr(2*r, w, v, t)
    # th_dfr_values.append((t, dfr))
    print(f"({t}, {round(dfr, 20)})")